In [2]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import os

# 파일 존재 여부 확인 함수
def check_file_exists(file_path):
    return os.path.isfile(file_path)

# 데이터 읽기 함수 (헤더 선택 포함)
def read_data(file_path):
    try:
        data = pd.read_csv(file_path, encoding='cp949', header=None)
        if messagebox.askyesno("헤더 확인", f"파일 {file_path}에 헤더가 있습니까?"):
            data = pd.read_csv(file_path, encoding='cp949')
        else:
            header_window = tk.Toplevel(root)
            header_window.title(f"헤더 선택 - {file_path}")

            label = tk.Label(header_window, text="첫 줄을 보고 각 열의 헤더를 입력하세요:")
            label.pack()

            entries = []
            first_row = data.iloc[0].tolist()

            for i, value in enumerate(first_row):
                frame = tk.Frame(header_window)
                frame.pack(fill='x')

                label = tk.Label(frame, text=f"열 {i+1}:", width=10)
                label.pack(side='left')

                entry = tk.Entry(frame)
                entry.pack(side='left', fill='x', expand=True)
                entry.insert(0, value)
                entries.append(entry)

            def set_headers():
                headers = [entry.get() for entry in entries]
                data.columns = headers
                data.drop(index=0, inplace=True)  # 첫 줄 제거
                header_window.destroy()
                data_loaded[file_path] = data

            button = tk.Button(header_window, text="확인", command=set_headers)
            button.pack()

            root.wait_window(header_window)
        return data
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return None

# 데이터 로드
data_loaded = {}

# GUI 설정
root = tk.Tk()
root.title("체납 및 입금 내역 조회")

# 파일 선택 함수
def upload_arrears_file():
    file_path = filedialog.askopenfilename(title="체납 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        arrears_data = read_data(file_path)
        data_loaded['arrears'] = arrears_data
        update_treeview(arrears_data, tree_arrears)

def upload_virtual_account_file():
    file_path = filedialog.askopenfilename(title="가상계좌 입금 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        virtual_account_data = read_data(file_path)
        data_loaded['virtual_account'] = virtual_account_data

def upload_cms_file():
    file_path = filedialog.askopenfilename(title="CMS 입금 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        cms_data = read_data(file_path)
        data_loaded['cms'] = cms_data

# 매핑 함수
def merge_data(arrears, deposit, withdrawal):
    merged = pd.merge(arrears, deposit, how='left', on=['청구마스터번호', '청구멤버번호', '계약번호'])
    merged = pd.merge(merged, withdrawal, how='left', on=['청구마스터번호', '청구멤버번호', '계약번호'])
    return merged

# 트리뷰 업데이트 함수
def update_treeview(data, tree):
    tree.delete(*tree.get_children())
    for _, row in data.iterrows():
        tree.insert("", "end", values=list(row))

# 검색 함수
def search():
    query = entry.get()
    filtered_data = merged_data[merged_data.apply(lambda row: row.astype(str).str.contains(query).any(), axis=1)]
    update_treeview(filtered_data, tree_merged)

# 버튼 및 트리뷰 구성
frame_buttons = tk.Frame(root)
frame_buttons.pack(pady=10)

button_upload_arrears = tk.Button(frame_buttons, text="체납 파일 업로드", command=upload_arrears_file)
button_upload_arrears.pack(side=tk.LEFT, padx=10)

button_upload_virtual_account = tk.Button(frame_buttons, text="가상계좌 입금 파일 업로드", command=upload_virtual_account_file)
button_upload_virtual_account.pack(side=tk.LEFT, padx=10)

button_upload_cms = tk.Button(frame_buttons, text="CMS 입금 파일 업로드", command=upload_cms_file)
button_upload_cms.pack(side=tk.LEFT, padx=10)

frame_search = tk.Frame(root)
frame_search.pack(pady=10)

entry = tk.Entry(frame_search, width=50)
entry.pack(side=tk.LEFT, padx=10)

button_search = tk.Button(frame_search, text="검색", command=search)
button_search.pack(side=tk.LEFT)

frame_treeviews = tk.Frame(root)
frame_treeviews.pack(expand=True, fill=tk.BOTH)

tree_arrears = ttk.Treeview(frame_treeviews, columns=columns, show='headings', height=10)
tree_arrears.pack(expand=True, fill=tk.BOTH)

tree_merged = ttk.Treeview(frame_treeviews, columns=columns, show='headings', height=10)
tree_merged.pack(expand=True, fill=tk.BOTH)

for col in columns:
    tree_arrears.heading(col, text=col)
    tree_arrears.column(col, width=100)
    tree_merged.heading(col, text=col)
    tree_merged.column(col, width=100)

# 데이터 매핑 및 트리뷰 업데이트
def merge_and_update_view():
    if 'arrears' in data_loaded and 'virtual_account' in data_loaded and 'cms' in data_loaded:
        merged_data = merge_data(data_loaded['arrears'], data_loaded['virtual_account'], data_loaded['cms'])
        update_treeview(merged_data, tree_merged)

button_merge_and_update = tk.Button(frame_buttons, text="데이터 매핑 및 업데이트", command=merge_and_update_view)
button_merge_and_update.pack(side=tk.LEFT, padx=10)

root.mainloop()

NameError: name 'columns' is not defined

In [3]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import os

# 필요한 컬럼
columns = ['청구마스터번호', '청구멤버번호', '계약번호', '청구맴버상호명', '수납', '체납잔액']

# 파일 존재 여부 확인 함수
def check_file_exists(file_path):
    return os.path.isfile(file_path)

# 데이터 읽기 함수 (헤더 선택 포함)
def read_data(file_path):
    try:
        data = pd.read_csv(file_path, encoding='cp949', header=None)
        if messagebox.askyesno("헤더 확인", f"파일 {file_path}에 헤더가 있습니까?"):
            data = pd.read_csv(file_path, encoding='cp949')
        else:
            header_window = tk.Toplevel(root)
            header_window.title(f"헤더 선택 - {file_path}")

            label = tk.Label(header_window, text="첫 줄을 보고 각 열의 헤더를 입력하세요:")
            label.pack()

            entries = []
            first_row = data.iloc[0].tolist()

            for i, value in enumerate(first_row):
                frame = tk.Frame(header_window)
                frame.pack(fill='x')

                label = tk.Label(frame, text=f"열 {i+1}:", width=10)
                label.pack(side='left')

                entry = tk.Entry(frame)
                entry.pack(side='left', fill='x', expand=True)
                entry.insert(0, value)
                entries.append(entry)

            def set_headers():
                headers = [entry.get() for entry in entries]
                data.columns = headers
                data.drop(index=0, inplace=True)  # 첫 줄 제거
                header_window.destroy()
                data_loaded[file_path] = data

            button = tk.Button(header_window, text="확인", command=set_headers)
            button.pack()

            root.wait_window(header_window)
        return data
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return None

# 데이터 로드
data_loaded = {}

# GUI 설정
root = tk.Tk()
root.title("체납 및 입금 내역 조회")

# 파일 선택 함수
def upload_arrears_file():
    file_path = filedialog.askopenfilename(title="체납 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        arrears_data = read_data(file_path)
        data_loaded['arrears'] = arrears_data
        update_treeview(arrears_data, tree_arrears)

def upload_virtual_account_file():
    file_path = filedialog.askopenfilename(title="가상계좌 입금 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        virtual_account_data = read_data(file_path)
        data_loaded['virtual_account'] = virtual_account_data

def upload_cms_file():
    file_path = filedialog.askopenfilename(title="CMS 입금 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        cms_data = read_data(file_path)
        data_loaded['cms'] = cms_data

# 매핑 함수
def merge_data(arrears, deposit, withdrawal):
    merged = pd.merge(arrears, deposit, how='left', on=['청구마스터번호', '청구멤버번호', '계약번호'])
    merged = pd.merge(merged, withdrawal, how='left', on=['청구마스터번호', '청구멤버번호', '계약번호'])
    return merged

# 트리뷰 업데이트 함수
def update_treeview(data, tree):
    tree.delete(*tree.get_children())
    for _, row in data.iterrows():
        tree.insert("", "end", values=list(row))

# 검색 함수
def search():
    query = entry.get()
    filtered_data = merged_data[merged_data.apply(lambda row: row.astype(str).str.contains(query).any(), axis=1)]
    update_treeview(filtered_data, tree_merged)

# 버튼 및 트리뷰 구성
frame_buttons = tk.Frame(root)
frame_buttons.pack(pady=10)

button_upload_arrears = tk.Button(frame_buttons, text="체납 파일 업로드", command=upload_arrears_file)
button_upload_arrears.pack(side=tk.LEFT, padx=10)

button_upload_virtual_account = tk.Button(frame_buttons, text="가상계좌 입금 파일 업로드", command=upload_virtual_account_file)
button_upload_virtual_account.pack(side=tk.LEFT, padx=10)

button_upload_cms = tk.Button(frame_buttons, text="CMS 입금 파일 업로드", command=upload_cms_file)
button_upload_cms.pack(side=tk.LEFT, padx=10)

frame_search = tk.Frame(root)
frame_search.pack(pady=10)

entry = tk.Entry(frame_search, width=50)
entry.pack(side=tk.LEFT, padx=10)

button_search = tk.Button(frame_search, text="검색", command=search)
button_search.pack(side=tk.LEFT)

frame_treeviews = tk.Frame(root)
frame_treeviews.pack(expand=True, fill=tk.BOTH)

tree_arrears = ttk.Treeview(frame_treeviews, columns=columns, show='headings', height=10)
tree_arrears.pack(expand=True, fill=tk.BOTH)

tree_merged = ttk.Treeview(frame_treeviews, columns=columns, show='headings', height=10)
tree_merged.pack(expand=True, fill=tk.BOTH)

for col in columns:
    tree_arrears.heading(col, text=col)
    tree_arrears.column(col, width=100)
    tree_merged.heading(col, text=col)
    tree_merged.column(col, width=100)

# 데이터 매핑 및 트리뷰 업데이트
def merge_and_update_view():
    if 'arrears' in data_loaded and 'virtual_account' in data_loaded and 'cms' in data_loaded:
        merged_data = merge_data(data_loaded['arrears'], data_loaded['virtual_account'], data_loaded['cms'])
        update_treeview(merged_data, tree_merged)

button_merge_and_update = tk.Button(frame_buttons, text="데이터 매핑 및 업데이트", command=merge_and_update_view)
button_merge_and_update.pack(side=tk.LEFT, padx=10)

root.mainloop()

KeyboardInterrupt: 

In [1]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import os

# 기본 파일 경로
arrears_file = 'D:\\체납\\이화영\\체납활동영업구역별 체납2회이상 활동대상.cs'
virtual_account_deposit_file = 'd:\\체납\\cs가상계좌입금.cs'
cms_withdrawal_file = 'd:\\체납\\5월 CMS출금.cs'

# 필요한 컬럼
columns = ['청구마스터번호', '청구멤버번호', '계약번호', '청구맴버상호명', '수납', '체납잔액']

# 파일 존재 여부 확인 함수
def check_file_exists(file_path):
    return os.path.isfile(file_path)

# 데이터 읽기 함수 (헤더 선택 포함)
def read_data(file_path):
    try:
        data = pd.read_csv(file_path, encoding='cp949', header=None)
        if messagebox.askyesno("헤더 확인", f"파일 {file_path}에 헤더가 있습니까?"):
            data = pd.read_csv(file_path, encoding='cp949')
        else:
            header_window = tk.Toplevel(root)
            header_window.title(f"헤더 선택 - {file_path}")

            label = tk.Label(header_window, text="첫 줄을 보고 각 열의 헤더를 입력하세요:")
            label.pack()

            entries = []
            first_row = data.iloc[0].tolist()

            for i, value in enumerate(first_row):
                frame = tk.Frame(header_window)
                frame.pack(fill='x')

                label = tk.Label(frame, text=f"열 {i+1}:", width=10)
                label.pack(side='left')

                entry = tk.Entry(frame)
                entry.pack(side='left', fill='x', expand=True)
                entry.insert(0, value)
                entries.append(entry)

            def set_headers():
                headers = [entry.get() for entry in entries]
                data.columns = headers
                data.drop(index=0, inplace=True)  # 첫 줄 제거
                header_window.destroy()
                data_loaded[file_path] = data

            button = tk.Button(header_window, text="확인", command=set_headers)
            button.pack()

            root.wait_window(header_window)
        return data
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return None

# 기본 데이터 로드
data_loaded = {}

if check_file_exists(arrears_file):
    arrears_data = read_data(arrears_file)
    data_loaded['arrears'] = arrears_data
else:
    print(f"Error: File {arrears_file} does not exist.")

if check_file_exists(virtual_account_deposit_file):
    virtual_account_deposit_data = read_data(virtual_account_deposit_file)
    data_loaded['virtual_account'] = virtual_account_deposit_data
else:
    print(f"Error: File {virtual_account_deposit_file} does not exist.")

if check_file_exists(cms_withdrawal_file):
    cms_withdrawal_data = read_data(cms_withdrawal_file)
    data_loaded['cms'] = cms_withdrawal_data
else:
    print(f"Error: File {cms_withdrawal_file} does not exist.")

# 데이터 매핑 함수
def merge_data(arrears, deposit, withdrawal):
    merged = pd.merge(arrears, deposit, how='left', on=['청구마스터번호', '청구멤버번호', '계약번호'])
    merged = pd.merge(merged, withdrawal, how='left', on=['청구마스터번호', '청구멤버번호', '계약번호'])
    return merged

# 초기 병합 데이터 생성
if 'arrears' in data_loaded and 'virtual_account' in data_loaded and 'cms' in data_loaded:
    merged_data = merge_data(data_loaded['arrears'], data_loaded['virtual_account'], data_loaded['cms'])

# GUI 설정
root = tk.Tk()
root.title("체납 및 입금 내역 조회")

# 파일 선택 함수
def upload_arrears_file():
    file_path = filedialog.askopenfilename(title="체납 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        arrears_data = read_data(file_path)
        data_loaded['arrears'] = arrears_data
        update_treeview(arrears_data, tree_arrears)
        update_merged_view()

def upload_virtual_account_file():
    file_path = filedialog.askopenfilename(title="가상계좌 입금 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        virtual_account_data = read_data(file_path)
        data_loaded['virtual_account'] = virtual_account_data
        update_merged_view()

def upload_cms_file():
    file_path = filedialog.askopenfilename(title="CMS 입금 파일 선택", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
    if file_path:
        cms_data = read_data(file_path)
        data_loaded['cms'] = cms_data
        update_merged_view()

# 트리뷰 업데이트 함수
def update_treeview(data, tree):
    tree.delete(*tree.get_children())
    for _, row in data.iterrows():
        tree.insert("", "end", values=list(row))

# 검색 함수
def search():
    query = entry.get()
    filtered_data = merged_data[merged_data.apply(lambda row: row.astype(str).str.contains(query).any(), axis=1)]
    update_treeview(filtered_data, tree_merged)

# 데이터 매핑 및 트리뷰 업데이트
def update_merged_view():
    if 'arrears' in data_loaded and 'virtual_account' in data_loaded and 'cms' in data_loaded:
        merged_data = merge_data(data_loaded['arrears'], data_loaded['virtual_account'], data_loaded['cms'])
        update_treeview(merged_data, tree_merged)

# 버튼 및 트리뷰 구성
frame_buttons = tk.Frame(root)
frame_buttons.pack(pady=10)

button_upload_arrears = tk.Button(frame_buttons, text="체납 파일 업로드", command=upload_arrears_file)
button_upload_arrears.pack(side=tk.LEFT, padx=10)

button_upload_virtual_account = tk.Button(frame_buttons, text="가상계좌 입금 파일 업로드", command=upload_virtual_account_file)
button_upload_virtual_account.pack(side=tk.LEFT, padx=10)

button_upload_cms = tk.Button(frame_buttons, text="CMS 입금 파일 업로드", command=upload_cms_file)
button_upload_cms.pack(side=tk.LEFT, padx=10)

frame_search = tk.Frame(root)
frame_search.pack(pady=10)

entry = tk.Entry(frame_search, width=50)
entry.pack(side=tk.LEFT, padx=10)

button_search = tk.Button(frame_search, text="검색", command=search)
button_search.pack(side=tk.LEFT)

frame_treeviews = tk.Frame(root)
frame_treeviews.pack(expand=True, fill=tk.BOTH)

tree_arrears = ttk.Treeview(frame_treeviews, columns=columns, show='headings', height=10)
tree_arrears.pack(expand=True, fill=tk.BOTH)

tree_merged = ttk.Treeview(frame_treeviews, columns=columns, show='headings', height=10)
tree_merged.pack(expand=True, fill=tk.BOTH)

for col in columns:
    tree_arrears.heading(col, text=col)
    tree_arrears.column(col, width=100)
    tree_merged.heading(col, text=col)
    tree_merged.column(col, width=100)

# 초기 데이터 표시
if 'arrears' in data_loaded:
    update_treeview(data_loaded['arrears'], tree_arrears)
if 'merged_data' in locals():
    update_treeview(merged_data, tree_merged)

root.mainloop()


Error: File d:\체납\cs가상계좌입금.cs does not exist.
Error: File d:\체납\5월 CMS출금.cs does not exist.


In [3]:
pip install pandas dash dash-bootstrap-components

Looking in indexes: https://pypi.org/simple/Note: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.5 MB 7.2 MB/s eta 0:00:01
   -- ------------------------------------- 0.5/7.5 MB 6.7 MB/s eta 0:00:02
   ---- ----------------------------------- 0.8/7.5 MB 5.9 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.5 MB 5.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.2/7.5 MB 5.3 MB/s eta 0:00:02
   ------- -------------------------------- 1.4/7.5 MB 5.1 MB/s eta 0:00:02
   -------- ------------------------------- 1.7/7.5 MB 5.1 MB/s eta 0:00:02
   ---------- ----------------------------- 1.9/7.5 MB 4.9 MB/s eta 0:00:02
   ----------- ---------------------------- 2.2/7.5 MB 4.9 MB/s eta 0:00:02
   ------------- -------------------------- 2.5/7.5 MB 4.9 MB/s eta 0:00:02
   -------------- ------------------------- 2.8/7.5 MB 4.8 M

In [8]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# 데이터 로드
def load_data():
    체납_data = pd.read_csv('D:\\체납\\이화영\\체납활동영업구역별 체납2회이상 활동대상.cs', encoding='euc-kr')
    가상계좌입금_data = pd.read_csv('D:\\체납\\가상계좌입금\\가상계좌병합.cs', encoding='euc-kr', header=1)
    CMS출금_data = pd.read_csv('D:\\체납\\5월 CMS출금\\5월_CMS출금_병합.cs', encoding='euc-kr')
    return 체납_data, 가상계좌입금_data, CMS출금_data

체납_data, 가상계좌입금_data, CMS출금_data = load_data()

# Dash 앱 설정
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("체납 활동 관리 시스템"), className="mb-2")
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Input(id='search-input', type='text', placeholder='검색...', className='mb-2'),
            dbc.Button("검색", id='search-button', n_clicks=0, className='mb-2'),
            dcc.Checklist(
                id='columns-checklist',
                options=[{'label': col, 'value': col} for col in 체납_data.columns],
                value=체납_data.columns.tolist(),
                labelStyle={'display': 'inline-block'}
            ),
            html.Div(id='table-container')
        ])
    ])
])

@app.callback(
    Output('table-container', 'children'),
    Input('search-button', 'n_clicks'),
    State('search-input', 'value'),
    State('columns-checklist', 'value')
)
def update_table(n_clicks, search_value, selected_columns):
    if n_clicks > 0:
        filtered_data = 체납_data
        if search_value:
            filtered_data = filtered_data[filtered_data.apply(lambda row: row.astype(str).str.contains(search_value).any(), axis=1)]
        return dbc.Table.from_dataframe(filtered_data[selected_columns], striped=True, bordered=True, hover=True)
    return ""

if __name__ == '__main__':
    app.run_server(debug=True)



UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x99 in position 222268: illegal multibyte sequence

In [9]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# 데이터 로드
def load_data():
    try:
        체납_data = pd.read_csv('D:\\체납\\이화영\\체납활동영업구역별 체납2회이상 활동대상.cs', encoding='euc-kr')
    except UnicodeDecodeError:
        체납_data = pd.read_csv('D:\\체납\\이화영\\체납활동영업구역별 체납2회이상 활동대상.cs', encoding='cp949')
   
    try:
        가상계좌입금_data = pd.read_csv('D:\\체납\\가상계좌입금\\가상계좌병합.cs', encoding='euc-kr', header=1)
    except UnicodeDecodeError:
        가상계좌입금_data = pd.read_csv('D:\\체납\\가상계좌입금\\가상계좌병합.cs', encoding='cp949', header=1)
   
    try:
        CMS출금_data = pd.read_csv('D:\\체납\\5월 CMS출금\\5월_CMS출금_병합.cs', encoding='euc-kr')
    except UnicodeDecodeError:
        CMS출금_data = pd.read_csv('D:\\체납\\5월 CMS출금\\5월_CMS출금_병합.cs', encoding='cp949')

    return 체납_data, 가상계좌입금_data, CMS출금_data

체납_data, 가상계좌입금_data, CMS출금_data = load_data()

# Dash 앱 설정
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("체납 활동 관리 시스템"), className="mb-2")
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Input(id='search-input', type='text', placeholder='검색...', className='mb-2'),
            dbc.Button("검색", id='search-button', n_clicks=0, className='mb-2'),
            dcc.Checklist(
                id='columns-checklist',
                options=[{'label': col, 'value': col} for col in 체납_data.columns],
                value=체납_data.columns.tolist(),
                labelStyle={'display': 'inline-block'}
            ),
            html.Div(id='table-container')
        ])
    ])
])

@app.callback(
    Output('table-container', 'children'),
    Input('search-button', 'n_clicks'),
    State('search-input', 'value'),
    State('columns-checklist', 'value')
)
def update_table(n_clicks, search_value, selected_columns):
    if n_clicks > 0:
        filtered_data = 체납_data
        if search_value:
            filtered_data = filtered_data[filtered_data.apply(lambda row: row.astype(str).str.contains(search_value).any(), axis=1)]
        return dbc.Table.from_dataframe(filtered_data[selected_columns], striped=True, bordered=True, hover=True)
    return ""

if __name__ == '__main__':
    app.run_server(debug=True)

In [14]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# 데이터 로드
def load_data():
    def read_csv_with_header(file_path, encoding, header):
        try:
            return pd.read_csv(file_path, encoding=encoding, header=header)
        except UnicodeDecodeError:
            return pd.read_csv(file_path, encoding='cp949', header=header)
   
    # 체납 데이터
    체납_data = read_csv_with_header('D:\\체납\\이화영\\체납활동영업구역별 체납2회이상 활동대상.cs', 'euc-kr', 0)
   
    # 가상계좌입금 데이터
    가상계좌입금_data = None
    for header in [0, 1]:
        try:
            가상계좌입금_data = read_csv_with_header('D:\\체납\\가상계좌입금\\가상계좌병합.cs', 'euc-kr', header)
            if '청구멤버번호' in 가상계좌입금_data.columns:
                break
        except Exception as e:
            pass
   
    # CMS출금 데이터
    CMS출금_data = None
    for header in [0, 1]:
        try:
            CMS출금_data = read_csv_with_header('D:\\체납\\5월 CMS출금\\5월_CMS출금_병합.cs', 'euc-kr', header)
            if '청구마스터번호' in CMS출금_data.columns:
                break
        except Exception as e:
            pass

    return 체납_data, 가상계좌입금_data, CMS출금_data

체납_data, 가상계좌입금_data, CMS출금_data = load_data()

# Dash 앱 설정
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("체납 활동 관리 시스템"), className="mb-2")
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("체납 데이터 키 선택"),
            dcc.Dropdown(
                id='체납-key-dropdown',
                options=[{'label': col, 'value': col} for col in 체납_data.columns],
                value=None,
                placeholder='체납 데이터 키 선택'
            ),
            html.Label("가상계좌 데이터 키 선택"),
            dcc.Dropdown(
                id='가상계좌-key-dropdown',
                options=[{'label': col, 'value': col} for col in 가상계좌입금_data.columns],
                value=None,
                placeholder='가상계좌 데이터 키 선택'
            ),
            html.Label("CMS 데이터 키 선택"),
            dcc.Dropdown(
                id='CMS-key-dropdown',
                options=[{'label': col, 'value': col} for col in CMS출금_data.columns],
                value=None,
                placeholder='CMS 데이터 키 선택'
            ),
            html.Label("미납관리지사 열 선택"),
            dcc.Dropdown(
                id='branch-column-dropdown',
                options=[{'label': col, 'value': col} for col in 체납_data.columns],
                value=None,
                placeholder='미납관리지사 열 선택'
            ),
            html.Label("영업구역 열 선택"),
            dcc.Dropdown(
                id='region-column-dropdown',
                options=[{'label': col, 'value': col} for col in 체납_data.columns],
                value=None,
                placeholder='영업구역 열 선택'
            ),
            dcc.Input(id='search-input', type='text', placeholder='검색...', className='mb-2'),
            dbc.Button("검색", id='search-button', n_clicks=0, className='mb-2'),
            dcc.Checklist(
                id='columns-checklist',
                options=[{'label': col, 'value': col} for col in 체납_data.columns],
                value=체납_data.columns.tolist(),
                labelStyle={'display': 'inline-block'}
            ),
            html.Div(id='table-container')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("체납 잔액 필터"),
            dcc.Dropdown(
                id='balance-filter',
                options=[
                    {'label': '100,000원 이상', 'value': 100000},
                    {'label': '150,000원 이상', 'value': 150000},
                    {'label': '200,000원 이상', 'value': 200000}
                ],
                multi=True,
                value=[]
            )
        ])
    ])
])

@app.callback(
    Output('table-container', 'children'),
    Input('search-button', 'n_clicks'),
    State('체납-key-dropdown', 'value'),
    State('가상계좌-key-dropdown', 'value'),
    State('CMS-key-dropdown', 'value'),
    State('branch-column-dropdown', 'value'),
    State('region-column-dropdown', 'value'),
    State('search-input', 'value'),
    State('columns-checklist', 'value'),
    State('balance-filter', 'value'),
    State('branch-filter', 'value'),
    State('region-filter', 'value')
)
def update_table(n_clicks, 체납_key, 가상계좌_key, CMS_key, branch_column, region_column, search_value, selected_columns, balance_filters, branch_filters, region_filters):
    if not (체납_key and 가상계좌_key and CMS_key):
        return html.Div("모든 키 값을 선택해 주세요.")

    # 데이터 매핑
    merged_data = 체납_data.merge(가상계좌입금_data, left_on=체납_key, right_on=가상계좌_key, how='left')\
                           .merge(CMS출금_data, left_on=체납_key, right_on=CMS_key, how='left')

    filtered_data = merged_data

    # 검색 필터
    if search_value:
        filtered_data = filtered_data[filtered_data.apply(lambda row: row.astype(str).str.contains(search_value).any(), axis=1)]
   
    # 잔액 필터
    if balance_filters:
        filtered_data = filtered_data[filtered_data['잔액'] >= min(balance_filters)]
   
    # 미납관리지사 필터
    if branch_filters and branch_column:
        filtered_data = filtered_data[filtered_data[branch_column].isin(branch_filters)]
   
    # 영업구역 필터
    if region_filters and region_column:
        filtered_data = filtered_data[filtered_data[region_column].isin(region_filters)]
   
    # 잔액 기준 정렬
    filtered_data = filtered_data.sort_values(by='잔액', ascending=False)
   
    # 선택한 컬럼만 표시
    table = dbc.Table.from_dataframe(filtered_data[selected_columns], striped=True, bordered=True, hover=True)
   
    return html.Div([
        dcc.ScrollView(table, style={'height': '400px', 'width': '100%', 'overflow': 'auto'})
    ])

if __name__ == '__main__':
    app.run_server(debug=True)

In [18]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# 데이터 로드 함수
def load_data():
    def read_csv_with_header(file_path, encoding, header):
        try:
            return pd.read_csv(file_path, encoding=encoding, header=header)
        except UnicodeDecodeError:
            return pd.read_csv(file_path, encoding='cp949', header=header)
   
    # 체납 데이터
    체납_data = read_csv_with_header('D:\\체납\\이화영\\체납활동영업구역별 체납2회이상 활동대상.cs', 'euc-kr', 0)
   
    # 가상계좌입금 데이터
    가상계좌입금_data = None
    for header in [0, 1]:
        try:
            가상계좌입금_data = read_csv_with_header('D:\\체납\\가상계좌입금\\가상계좌병합.cs', 'euc-kr', header)
            if '청구멤버번호' in 가상계좌입금_data.columns:
                break
        except Exception as e:
            pass
   
    # CMS출금 데이터 (헤더가 1행)
    try:
        CMS출금_data = read_csv_with_header('D:\\체납\\5월 CMS출금\\5월_CMS출금_병합.cs', 'euc-kr', header=0)
    except Exception as e:
        CMS출금_data = read_csv_with_header('D:\\체납\\5월 CMS출금\\5월_CMS출금_병합.cs', 'cp949', header=0)

    return 체납_data, 가상계좌입금_data, CMS출금_data

체납_data, 가상계좌입금_data, CMS출금_data = load_data()

# 데이터 컬럼 옵션 생성 함수
def generate_column_options(data):
    if data is not None:
        return [{'label': col, 'value': col} for col in data.columns]
    return []

# Dash 앱 설정
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# 앱 레이아웃
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("체납 활동 관리 시스템"), className="mb-2")
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("데이터셋 선택"),
            dcc.Checklist(
                id='dataset-checklist',
                options=[
                    {'label': '체납 데이터', 'value': '체납'},
                    {'label': '가상계좌 데이터', 'value': '가상계좌'},
                    {'label': 'CMS 데이터', 'value': 'CMS'}
                ],
                value=['체납', '가상계좌', 'CMS'],
                labelStyle={'display': 'inline-block'}
            ),
            html.Label("체납 데이터 키 선택"),
            dcc.Dropdown(
                id='체납-key-dropdown',
                options=generate_column_options(체납_data),
                value=None,
                placeholder='체납 데이터 키 선택'
            ),
            html.Label("가상계좌 데이터 키 선택"),
            dcc.Dropdown(
                id='가상계좌-key-dropdown',
                options=generate_column_options(가상계좌입금_data),
                value=None,
                placeholder='가상계좌 데이터 키 선택'
            ),
            html.Label("CMS 데이터 키 선택"),
            dcc.Dropdown(
                id='CMS-key-dropdown',
                options=generate_column_options(CMS출금_data),
                value=None,
                placeholder='CMS 데이터 키 선택'
            ),
            html.Label("미납관리지사 열 선택"),
            dcc.Dropdown(
                id='branch-column-dropdown',
                options=generate_column_options(체납_data),
                value=None,
                placeholder='미납관리지사 열 선택'
            ),
            html.Label("영업구역 열 선택"),
            dcc.Dropdown(
                id='region-column-dropdown',
                options=generate_column_options(체납_data),
                value=None,
                placeholder='영업구역 열 선택'
            ),
            dcc.Input(id='search-input', type='text', placeholder='검색...', className='mb-2'),
            dbc.Button("검색", id='search-button', n_clicks=0, className='mb-2'),
            dcc.Checklist(
                id='columns-checklist',
                options=generate_column_options(체납_data),
                value=체납_data.columns.tolist() if 체납_data is not None else [],
                labelStyle={'display': 'inline-block'}
            ),
            html.Div(id='table-container')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("체납 잔액 필터"),
            dcc.Dropdown(
                id='balance-filter',
                options=[
                    {'label': '100,000원 이상', 'value': 100000},
                    {'label': '150,000원 이상', 'value': 150000},
                    {'label': '200,000원 이상', 'value': 200000}
                ],
                multi=True,
                value=[]
            )
        ])
    ])
])

# 콜백 함수
@app.callback(
    Output('table-container', 'children'),
    Input('search-button', 'n_clicks'),
    State('dataset-checklist', 'value'),
    State('체납-key-dropdown', 'value'),
    State('가상계좌-key-dropdown', 'value'),
    State('CMS-key-dropdown', 'value'),
    State('branch-column-dropdown', 'value'),
    State('region-column-dropdown', 'value'),
    State('search-input', 'value'),
    State('columns-checklist', 'value'),
    State('balance-filter', 'value'),
    State('branch-filter', 'value'),
    State('region-filter', 'value')
)
def update_table(n_clicks, selected_datasets, 체납_key, 가상계좌_key, CMS_key, branch_column, region_column, search_value, selected_columns, balance_filters, branch_filters, region_filters):
    if not selected_datasets:
        return html.Div("데이터셋을 선택해주세요.")

    merged_data = pd.DataFrame()

    # 데이터셋 선택에 따른 병합
    if '체납' in selected_datasets:
        merged_data = 체납_data.copy()
        if '가상계좌' in selected_datasets and 체납_key and 가상계좌_key:
            merged_data = merged_data.merge(가상계좌입금_data, left_on=체납_key, right_on=가상계좌_key, how='left')
        if 'CMS' in selected_datasets and 체납_key and CMS_key:
            merged_data = merged_data.merge(CMS출금_data, left_on=체납_key, right_on=CMS_key, how='left')
    elif '가상계좌' in selected_datasets:
        merged_data = 가상계좌입금_data.copy()
        if 'CMS' in selected_datasets and 가상계좌_key and CMS_key:
            merged_data = merged_data.merge(CMS출금_data, left_on=가상계좌_key, right_on=CMS_key, how='left')
    elif 'CMS' in selected_datasets:
        merged_data = CMS출금_data.copy()

    filtered_data = merged_data

    # 검색 필터
    if search_value:
        filtered_data = filtered_data[filtered_data.apply(lambda row: row.astype(str).str.contains(search_value).any(), axis=1)]
   
    # 잔액 필터
    if balance_filters and '잔액' in filtered_data.columns:
        filtered_data = filtered_data[filtered_data['잔액'] >= min(balance_filters)]
   
    # 미납관리지사 필터
    if branch_filters and branch_column and branch_column in filtered_data.columns:
        filtered_data = filtered_data[filtered_data[branch_column].isin(branch_filters)]
   
    # 영업구역 필터
    if region_filters and region_column and region_column in filtered_data.columns:
        filtered_data = filtered_data[filtered_data[region_column].isin(region_filters)]
   
    # 잔액 기준 정렬
    if '잔액' in filtered_data.columns:
        filtered_data = filtered_data.sort_values(by='잔액', ascending=False)
   
    # 선택한 컬럼만 표시
    if selected_columns:
        filtered_data = filtered_data[selected_columns]
   
    table = dbc.Table.from_dataframe(filtered_data, striped=True, bordered=True, hover=True)
   
    return table

if __name__ == '__main__':
    app.run_server(debug=True)